In [1]:
import os
os.sys.path.append("..")
from src.vllm import llm_chat, message_template
import time
from openai import OpenAI
from glob import glob
import json
from concurrent.futures import ThreadPoolExecutor, as_completed

# python -m vllm.entrypoints.openai.api_server --model meta-llama/Meta-Llama-3-70B-Instruct --tensor-parallel-size=2 --disable-log-requests

In [53]:
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"
# model_name = "llama3-70b"
model = "meta-llama/Meta-Llama-3-70B-Instruct"

if model == "meta-llama/Meta-Llama-3-70B-Instruct":
    client = OpenAI(
        api_key=openai_api_key,
        base_url=openai_api_base,
    )

def llm_chat(messages: list[dict], guided_json=None):
    print(messages, guided_json)
    chat_response = client.chat.completions.create(
        model=model,
        messages=messages,
        extra_body={
            "guided_json": guided_json
        }
    )
    return chat_response.choices[0].message.content


In [55]:
def clean_data(data):
    cleaned_data = {}
    for k, v in data.items():
        if type(v) == '' and v == '':
            continue
        elif type(v) == list and len(v) == 0:
            continue
        cleaned_data[k] = v
    return cleaned_data

def format_data_to_string(data):
    result = ""
    if data.get("share_price", None) is not None:
        result += "Today's share price: " + str(data["share_price"])
    for k, v in data.items():
        if k != 'share_price':
            result += f"\nToday's <{k}>: " + str(v)
    return result

In [63]:
prompt = "Given today's share price and stock related summary, predict the next day's share price and summary"

result_schema = {
  "type": "object",
  "properties": {
    "summary": {
      "type": "string",
      "description": "Summary of next day's article"
    },
    "share_price": {
        "type": "integer",
        "description": "Next day's share price prediction"
    }
  },
    "required": [
    "summary",
    "share_price",
  ]
}

In [75]:
responses = []
paths = sorted(glob("/data/kai/forecasting/data/formatted_v0.2/AMD/*.json"))

def test_text_time(path):
    with open(path, 'r') as f:
        data = clean_data(json.load(f))

    content = format_data_to_string(data)
    message = message_template(prompt, content)
    response = llm_chat(message, guided_json=result_schema)
    return response

with ThreadPoolExecutor(max_workers=20) as executor:
    futures = [executor.submit(test_text_time, path) for path in paths]
    results = [future.result() for future in as_completed(futures)]

[{'role': 'system', 'content': "Given today's share price and stock related summary, predict the next day's share price and summary"}, {'role': 'user', 'content': "Today's share price: 111.98\nToday's <notable_company_specific_news>: ['Intel Corp forecast second-quarter revenue and profit below Wall Street expectations due to demand weakness in PCs and supply-chain uncertainty from COVID-19 lockdowns in China.']\nToday's <summary>: Intel Corp forecasts a gloomy quarter due to supply-chain woes, citing demand weakness in PCs and COVID-19 lockdowns in China, leading to a fall in its shares."}] {'type': 'object', 'properties': {'summary': {'type': 'string', 'description': "Summary of next day's article"}, 'share_price': {'type': 'integer', 'description': "Next day's share price prediction"}}, 'required': ['summary', 'share_price']}
[{'role': 'system', 'content': "Given today's share price and stock related summary, predict the next day's share price and summary"}, {'role': 'user', 'conten